# CSE 258, Fall 2019: Homework 3
**You’ll probably want to implement your solution by modifying the baseline code provided.**   
Files: 
* http://cseweb.ucsd.edu/classes/fa19/cse258-a/files/assignment1.tar.gz   

Kaggle:
* https://inclass.kaggle.com/c/cse158258-fa19-read-prediction
* (258 only) https://inclass.kaggle.com/c/cse258-fa19-rating-prediction

# Tasks (Read prediction)   
Since we don’t have access to the test labels, we’ll need to simulate validation/test sets of our own.    
So, let’s split the training data (‘train Interactions.csv.gz’) as follows:
1. Reviews 1-190,000 for training
2. Reviews 190,001-200,000 for validation
3. Upload to Kaggle for testing only when you have a good model on the validation set. This will save you time (since Kaggle can take several minutes to return results), and prevent you from exceeding your daily submission limit.

In [1]:
import gzip
from collections import defaultdict
import random
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy
from sklearn.linear_model import LogisticRegression

In [53]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)
    
def readCSV(path):
    f = gzip.open(path, 'rt')
    header = f.readline()
    for l in f:
        yield l.strip().split(',')
        
def accuracy(predictions, labels):
    predictions, labels = np.array(predictions), np.array(labels)
    return sum(predictions == labels) / len(predictions)

def most_popular_percentile(mostPopular, percentile):
    return1 = set()
    count = 0
    for b_count, b in mostPopular:
        count += b_count
        return1.add(b)
        if count > percentile * totalRead: break
    return return1

# Extra utils for this task
def cosine_sim(s1,s2):
    numer = len(s1.intersection(s2))
    denom = len(s1) * len(s2) + 10**(-8)
    return numer / denom 
    
def smallest_cosine(user, book):
    users = usersPerBook[book]
    b_mark = bookPerUser[user] # Books that user has read
    angels = []
    for book2 in b_mark:
        if book2 == book:
            continue
        angel = cosine_sim(users, usersPerBook[book2])
        angels.append((angel, book2))
    angels.sort(reverse=True)
    if len(angels) == 0:
        return 0
    return angels[0][0]

def best_pearson_corr(user, book):
    # users who rated both books is what we want
    avg_rate_book = sum([x[1] for x in reviewsPerBook[book]]) / (len(reviewsPerBook[book]) + 10**(-8))
    
    reviews_on_book = reviewsPerBook[book]
    reviews_on_book_dict = dict(reviews_on_book)
    
    users_who_rated_book = set([x[0] for x in reviews_on_book])

    books_user_have_read = bookPerUser[user]
    similarities = []
    for book2 in books_user_have_read:
        avg_rate_book2 = sum([x[1] for x in reviewsPerBook[book2]])/ (len(reviewsPerBook[book2]) + 10**(-8))
        
        reviews_on_book2 = reviewsPerBook[book2]
        reviews_on_book2_dict = dict(reviews_on_book2)
        
        users_who_rated_book2 = set([x[0] for x in reviews_on_book2])

        users_who_rated_both_books = users_who_rated_book.intersection(users_who_rated_book2)
        
        # Pearson Correlation happens here:
        numer = 0
        denom1 = 0
        denom2 = 0
        for user2 in users_who_rated_both_books:

            a = reviews_on_book_dict[user2] - avg_rate_book
            b = reviews_on_book2_dict[user2] - avg_rate_book2
            numer += a * b
            
            denom1 += a**2
            denom2 += b**2
        
        denom1 = np.sqrt(denom1)
        denom2 = np.sqrt(denom2)
        similarities.append((numer/(denom1*denom2 + 10**(-8)), book2))
    similarities.sort(reverse=True)
    return similarities[0][0]

def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

def best_jacc(user, book):
    users = usersPerBook[book]
    b_mark = bookPerUser[user]
    similarities = []
    for book2 in b_mark:
        if book2 == book:
            continue
        # compute sim between book and book2
        sim = Jaccard(users, usersPerBook[book2])
        similarities.append(sim)
    similarities.sort(reverse=True)
    return similarities
        

In [54]:
#data = [line[:2] + [1] for line in readCSV("train_Interactions.csv.gz")] # 1 is the label saying it is read.
data = [line for line in readCSV("train_Interactions.csv.gz")]

# Extend validation set

In [55]:
Xy_train = data[:190000]
Xy_valid = data[190000:]
# First get overview of what books each user have read, and what what user a book has been read by.
usersPerBook = defaultdict(set)
bookPerUser = defaultdict(set)
for line in data:
    userID, bookID, rating = line
    usersPerBook[bookID].add(userID)
    bookPerUser[userID].add(bookID)

# Randomly ad some negative samples to the validation set
negative_samples = []
available_books = usersPerBook.keys()
for user, book, rating in Xy_valid:
    #print(user,book)
    random_book = random.choice(list(available_books))
    while random_book in bookPerUser[user]:
        random_book = random.choice(list(available_books))
    new_data = [user, random_book, 0]
    negative_samples.append(new_data)
Xy_valid += negative_samples # Add the negative data
random.shuffle(Xy_valid)

Xtrain, ytrain = [d[:2] for d in Xy_train], [int(d[2]) for d in Xy_train]
Xvalid, yvalid = [d[:2] for d in Xy_valid], [int(d[2]) for d in Xy_valid]

# Predict

In [109]:
### Would-read baseline: just rank which books are popular and which are not, and return '1' if a book is among the top-ranked
# BASELINE: ACC 0.6576

bookCount = defaultdict(int)
totalRead = 0
for user,book in Xtrain:
    bookCount[book] += 1
    totalRead += 1
mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort(reverse=True)
#print(mostPopular)
top_10 = most_popular_percentile(mostPopular, 0.20)
top_50 = most_popular_percentile(mostPopular, 0.55)





TP, FP = 0, 0
jacc_threshold = 0.012
predictions = []
for (user, book), rating in zip(Xvalid, yvalid):
    # Poppularity
    istop10 = book in top_10
    istop50 = book in top_50
    # Jaccard
    jaccard_sims = best_jacc(user, book)
    jaccs_over_zero = list(filter(lambda x: x > 0,jaccard_sims))
    jacc_avg = sum(jaccs_over_zero) / (len(jaccs_over_zero) + 10**(-8))

    pred = 0
    if istop10:
        pred = 1
        # Acc 0.80 here
        #print(pred, rating)
    elif jacc_avg > jacc_threshold:
        # Acc 0.65 here
        pred = 1
        if pred == rating:
            TP += 1
        else:
            FP += 1
    
        
    
    # if pred = 0 -> acc 0.80+

        
    predictions.append(pred)

print(TP, FP)
yvalid = list(map(lambda x: int(x>0), yvalid))
print("Accuracy: {}".format(accuracy(predictions, yvalid)))

6980 4000
Accuracy: 0.7419


0 0.2438510862283472
1 0.4791666666666667
2 0.45698924731182794
3 0.4681528662420382
4 0.529674003900808
5 0.5972299168975069


This showns that for higher ratings, it is a higher changce for the book to be in the popularpercentile

## Task 4
Improve the above predictor by incorporating both a Jaccard-based threshold and a popularity based threshold. Report the performance on your validation set (1 mark)

In [12]:
# For pearson
data = [line for line in readCSV("train_Interactions.csv.gz")]
Xy_train_per, Xy_valid_per = data[:190000], data[190000:]
Xtrain_per, ytrain_per = [x[:2] for x in Xy_train_per], [int(x[-1]) for x in Xy_train_per]
Xvalid_per, yvalid_per = [x[:2] for x in Xy_valid_per], [int(x[-1]) for x in Xy_valid_per]

reviewsPerUser = defaultdict(list)
reviewsPerBook = defaultdict(list)

for user, book, rating in Xy_train_per:
    rating = int(rating)
    reviewsPerUser[user].append((book, rating))
    reviewsPerBook[book].append((user, rating))

In [59]:
# For each book make a featureset [isPopular, Jaccard] and learn use a Classifier from sklearn to learn the params
popularity_th = 0.55
popularPercentile = most_popular_percentile(mostPopular, popularity_th) 


# Calculate features for the classifier to find weights
def feature(user, book):
    popular = 1 if book in popularPercentile else 0
    jac_sim = best_jacc(user, book)
    cos = smallest_cosine(user,book)
    pearson_corr = best_pearson_corr(user, book)
    return [1, jac_sim, pearson_corr]

newX = [feature(user, book) for user, book in Xvalid]

In [81]:
# Make a model that can compute the weights for me
model = LogisticRegression(solver="lbfgs", fit_intercept=False, class_weight="balanced")
model.fit(newX, yvalid)
theta = model.coef_[0]
theta

array([-0.93479015,  7.32214529,  1.12944214])

# Upload Test Results
## Kaggle Username: kristogj

In [72]:
predictions = open("predictions_Read.txt", 'w')
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    user, book = l.strip().split('-')
    pred = predict(user, book, theta)
    predictions.write(user + '-' + book + ",{}\n".format(pred))
predictions.close()

# (CSE 258 only) Tasks (Rating prediction)

Let’s start by building our training/validation sets much as we did for the first task. This time building a validation set is more straightforward: you can simply use part of the data for validation, and do not need to randomly sample non-read users/books.

In [69]:
data = [line for line in readCSV("train_Interactions.csv.gz")]
Xy_train, Xy_valid = data[:190000], data[190000:]
Xtrain, ytrain = [x[:2] for x in Xy_train], [int(x[-1]) for x in Xy_train]
Xvalid, yvalid = [x[:2] for x in Xy_valid], [int(x[-1]) for x in Xy_valid]

## Task 9
Fit a predictor of the form

$rating(user, item) = \alpha + \beta_{user} + \beta_{item}$


by fitting the mean and the two bias terms as described in the lecture notes. Use a regularization
parameter of λ = 1. Report the MSE on the validation set (1 mark).

In [70]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [73]:
reviewsPerUser = defaultdict(list)
reviewsPerBook = defaultdict(list)

for user, book, rating in Xy_train:
    rating = int(rating)
    reviewsPerUser[user].append(rating)
    reviewsPerBook[book].append(rating)

ratingMean = sum(ytrain) / len(ytrain)

N = len(ytrain)
nUsers = len(reviewsPerUser)
nBooks = len(reviewsPerBook)
users = list(reviewsPerUser.keys())
books = list(reviewsPerBook.keys())

lamb = 1
alpha = ratingMean
userBiases = defaultdict(float)
bookBiases = defaultdict(float)

In [74]:
def prediction(user, book):
    return alpha + userBiases[user] + bookBiases[book]

In [75]:
def unpack(theta):
    global alpha
    global userBiases
    global bookBiases
    alpha = theta[0]
    userBiases = defaultdict(float)
    bookBiases = defaultdict(float)
    for user, t in zip(users, theta[1:nUsers+1]):
        userBiases[user] = t
    for book, t in zip(books, theta[1+nUsers:]):
        bookBiases[book] = t

def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(user, book) for user, book in Xtrain]
    cost = MSE(predictions, ytrain)
    #print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for b in bookBiases:
        cost += lamb*bookBiases[b]**2
    return cost

def derivative(theta, labels, lamb):
    unpack(theta)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dBookBiases = defaultdict(float)
    for user, book, rating in Xy_train:
        rating = int(rating)
        pred = prediction(user, book)
        diff = pred - rating
        dalpha += 2/N*diff
        dUserBiases[user] += 2/N*diff
        dBookBiases[book] += 2/N*diff
    for u in userBiases:
        dUserBiases[user] += 2*lamb*userBiases[user]
    for i in bookBiases:
        dBookBiases[book] += 2*lamb*bookBiases[book]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dBookBiases[b] for b in books]
    return np.array(dtheta)

In [76]:
x, f, d = scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nBooks),
                             derivative, args = (ytrain, lamb))

In [77]:
predictions = []
for user, book in Xvalid:
    predictions.append(prediction(user, book))

mse = MSE(predictions, yvalid)
print("MSE: {}".format(mse))

MSE: 1.4907803977377663


## Task 10
Report the user and book IDs that have the largest and smallest values of β (1 mark).

In [78]:
# Code here
userBiases_items = list(userBiases.items())
userBiases_items.sort(key=lambda x: x[1], reverse=True)
print("\n Largest user bias: {} \n Smallest user bias: {}".format(userBiases_items[0], userBiases_items[-1]))

bookBiases_items = list(bookBiases.items())
bookBiases_items.sort(key=lambda x: x[1], reverse=True)
print("\n Largest book bias: {} \n Smallest book bias: {}".format(bookBiases_items[0], bookBiases_items[-1]))


 Largest user bias: ('u92864068', 0.0004044337058247039) 
 Smallest user bias: ('u11591742', -0.0015810150664586793)

 Largest book bias: ('b76915592', 0.0008308782940987173) 
 Smallest book bias: ('b57299824', -0.0002723172505094118)


## Task 11
Find a better value of λ using your validation set. Report the value you chose, its MSE, and upload your solution to Kaggle by running it on the test data (1 mark).
# Kaggle Username: kristogj

In [79]:
lamb = 0.00002
x, f, d = scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nBooks),
                             derivative, args = (ytrain, lamb))

I found the right value for lamb by looping over different values of lamb and observing how the mse was chaning. 

In [80]:
predictions = []
for user, book in Xvalid:
    predictions.append(prediction(user, book))

mse = MSE(predictions, yvalid)
print("MSE: {}".format(mse))

MSE: 1.1742062925437837


In [81]:
predictions = open("predictions_Rating.txt", 'w')
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    user, book = l.strip().split('-')
    pred = str(prediction(user, book))
    predictions.write(user + '-' + book + ',' + pred + '\n')
predictions.close()